# Get FASTA (i.e. total theoretical) peptides
    Author: Anima Sutradhar
    Project: Peptide detectability prediction to improve protein identification in mass spectrometry using machine learning.

## Notebook summary:
1. Import UniProtKB/SwissProt FASTA file (Homo sapiens proteome).
2. Save FASTA file as TSV and reformat.
3. Create in silico tryptic digest code.
4. Perform in silico tryptic digest (with 0 missed cleavages) on reformatted UniProtKB/SwissProt protein sequences.
5. Reformat digested peptide sequences.
6. Check for total undigested proteins and remove.
7. Export digested peptide dataset as TSV.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
from Bio import SeqIO
import csv

### 1. Import UniProtKB/SwissProt FASTA file (Homo sapiens proteome)

In [2]:
fasta = pd.read_table('../data/uniprot-swissprot-human.fasta', header=None)
fasta.head()

,0
0,>sp|Q66K14|TBC9B_HUMAN TBC1 domain family memb...
1,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...
2,ILHQTQDSQVYWTVACGSSRKEITKHWEWLENNLLQTLSIFDSEED...
3,EENKNLQPQGDEDPGKFKEAELKMRKQFGMPEGEKLVNYYSCSYWK...
4,HLCFYSFLLGKEVSLVVQWVDITRLEKNATLLFPESIRVDTRDQEL...


In [3]:
# check dimensions
fasta.shape

(219719, 1)

#### Verification: check number of protein sequences in FASTA file (using BioPython)

In [4]:
records = list(SeqIO.parse('../data/uniprot-swissprot-human.fasta', 'fasta'))
print("Total sequences: %i" % len(records))

Total sequences: 20380


### 2. Save FASTA file as TSV and reformat

In [5]:
with open('../data/fasta.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    tsvfile.write("Sequence\tFasta headers\n")
    for record in SeqIO.parse("../data/uniprot-swissprot-human.fasta", "fasta"):
        writer.writerow([record.seq, record.id])

In [6]:
fasta_proteins = pd.read_table('../data/fasta.tsv')
fasta_proteins.head()

,Sequence,Fasta headers
0,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,sp|Q66K14|TBC9B_HUMAN
1,MEFTASPKPQLSSRANAFSIAALMSSGGSKEKEATENTIKPLEQFV...,sp|Q9UMR3|TBX20_HUMAN
2,MAPVRRSAKWRPGGIEARGEGVSTVGYRNKNVRQKTWRPNHPQAFV...,sp|Q9P031|TAP26_HUMAN
3,MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGD...,sp|Q6PEY2|TBA3E_HUMAN
4,MSRPRKRLAGTSGSDKGLSGKRTKTENSGEALAKVEDSNPQKTSAT...,sp|Q9P016|THYN1_HUMAN


In [7]:
# verification: check dimensions before formatting
fasta_proteins.shape

(20380, 2)

In [8]:
# extract protein IDs from FASTA headers, reorder columns
fasta_proteins['Protein'] = fasta_proteins['Fasta headers'].str.split('|').str[1]
fasta_proteins.drop(['Fasta headers'], axis=1)
reorder_col = ['Protein', 'Sequence']
fasta_proteins = fasta_proteins.reindex(columns=reorder_col)
fasta_proteins.head()

,Protein,Sequence
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...
1,Q9UMR3,MEFTASPKPQLSSRANAFSIAALMSSGGSKEKEATENTIKPLEQFV...
2,Q9P031,MAPVRRSAKWRPGGIEARGEGVSTVGYRNKNVRQKTWRPNHPQAFV...
3,Q6PEY2,MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGD...
4,Q9P016,MSRPRKRLAGTSGSDKGLSGKRTKTENSGEALAKVEDSNPQKTSAT...


In [9]:
fasta_proteins.shape

(20380, 2)

In [10]:
fasta_proteins.to_csv("../data/fasta_proteins.tsv", sep='\t', index=False)

### 3. Create in silico tryptic digest code

In [11]:
# This code was kindly provided by Esteban Gea on 20/04/2021. I have adapted the regex pattern to
# perform a strict tryptic digest and moved the count variable to allow for the correct number of
# missed cleavages. Values for 'misCleavage', 'min_peptide_length' and 'max_peptide_length' have
# been modified to match MaxQuant settings used for this experiment.

def digest(seq, returnStart):
    """Perform tryptic digest given a protein sequence. Returns the digested peptide sequences
    (assuming 0 missed cleavages).
    :param:
        seq -- protein sequence.
        returnStart -- return sequence position of the digested peptide: 0 for TRUE, 1 for FALSE.
    :return: digested peptides, along with their sequence position (if specified).
    """
    
    pattern = "(.(?:(?<![KR](?!P)).)*)"
    frags = list(filter(None, re.findall(pattern, seq)))
    misCleavage = 0
    min_peptide_length = 7
    max_peptide_length = 51
    peptides = []

    for i in range(0, len(frags)):
        count = 0
        if i == 0:
            start = 0
        else:
            start = len("".join(frags[:i]))

        if (len(frags[i]) >= min_peptide_length) & (len(frags[i]) <= max_peptide_length):
            if returnStart:
                peptides.append({"sequence": frags[i], "start": start})
            else:
                peptides.append(frags[i])

        if i != len(frags) - 1:
            for j in range(i + 1, len(frags)):
                if count < misCleavage:
                    count = count + 1
                    pep = "".join(frags[i:j + 1])
                    if (len(pep) >= min_peptide_length) & (len(pep) <= max_peptide_length):
                        if returnStart:
                            peptides.append({"sequence": pep, "start": start})
                        else:
                            peptides.append(pep)
                    elif len(pep) > 40:
                        break
                else:
                    break

    return peptides

#### Verification of in silico tryptic digest code

In [12]:
# check tryptic digest regex code works correctly (cleaves at K/R but not before P).
# check missed cleavages in code works correctly (KKKK+ and RRRR+ in peptide sequences should not exist).

digest("AAAAAAAAAVSRRRKAEYPRRRRRSSPSARPPDVPGQQPQAAKP", 0)
# expected result: total 2 sequences when miscleavage = 0

['AAAAAAAAAVSR', 'SSPSARPPDVPGQQPQAAKP']

### 4. Perform in silico tryptic digest on reformatted UniProtKB/SwissProt protein sequences

In [13]:
fasta_proteins = pd.read_table('../data/fasta_proteins.tsv', header=0)
fasta_proteins.head()

,Protein,Sequence
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...
1,Q9UMR3,MEFTASPKPQLSSRANAFSIAALMSSGGSKEKEATENTIKPLEQFV...
2,Q9P031,MAPVRRSAKWRPGGIEARGEGVSTVGYRNKNVRQKTWRPNHPQAFV...
3,Q6PEY2,MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGD...
4,Q9P016,MSRPRKRLAGTSGSDKGLSGKRTKTENSGEALAKVEDSNPQKTSAT...


In [14]:
fasta_peptides = fasta_proteins
fasta_peptides['Peptide'] = fasta_peptides.apply(lambda row : digest(row['Sequence'], 0), axis = 1)
fasta_peptides.head()

,Protein,Sequence,Peptide
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,"[MWLSPEEVLVANALWVTER, ANPFFVLQR, GGGLTGLLVGTLD..."
1,Q9UMR3,MEFTASPKPQLSSRANAFSIAALMSSGGSKEKEATENTIKPLEQFV...,"[MEFTASPKPQLSSR, ANAFSIAALMSSGGSK, EATENTIKPLE..."
2,Q9P031,MAPVRRSAKWRPGGIEARGEGVSTVGYRNKNVRQKTWRPNHPQAFV...,"[WRPGGIEAR, GEGVSTVGYR, TWRPNHPQAFVGSVR, EGQGF..."
3,Q6PEY2,MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGD...,"[ECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDK, TIGGG..."
4,Q9P016,MSRPRKRLAGTSGSDKGLSGKRTKTENSGEALAKVEDSNPQKTSAT...,"[LAGTSGSDK, TENSGEALAK, VEDSNPQK, NLSSHWLMK, F..."


#### Verification: check if expected number of proteins are present following digest

In [15]:
fasta_peptides.shape

(20380, 3)

### 5. Reformat digested peptide sequences

In [16]:
# explode list contents into separate rows (one sequence per row)
fasta_peptides = fasta_peptides.explode('Peptide')
fasta_peptides.head()

,Protein,Sequence,Peptide
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,MWLSPEEVLVANALWVTER
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,ANPFFVLQR
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,GGGLTGLLVGTLDVVLDSSAR
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,ILHQTQDSQVYWTVACGSSR
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,HWEWLENNLLQTLSIFDSEEDITTFVK


In [17]:
# get sequence length for each protein, add values to a "Length" column
# this is to calculate protein quantitation metrics later on
fasta_peptides["Length"] = fasta_peptides["Sequence"].str.len()
fasta_peptides

,Protein,Sequence,Peptide,Length
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,MWLSPEEVLVANALWVTER,1250
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,ANPFFVLQR,1250
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,GGGLTGLLVGTLDVVLDSSAR,1250
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,ILHQTQDSQVYWTVACGSSR,1250
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...,HWEWLENNLLQTLSIFDSEEDITTFVK,1250
...,...,...,...,...
20379,P16234,MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFS...,VDSDNAYIGVTYK,1089
20379,P16234,MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFS...,DWEGGLDEQR,1089
20379,P16234,MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFS...,LSADSGYIIPLPDIDPVPEEEDLGK,1089
20379,P16234,MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFS...,HSSQTSEESAIETGSSSSTFIK,1089


In [18]:
# remove sequence column
fasta_peptides = fasta_peptides.drop(['Sequence'], axis=1)
fasta_peptides.head()

,Protein,Peptide,Length
0,Q66K14,MWLSPEEVLVANALWVTER,1250
0,Q66K14,ANPFFVLQR,1250
0,Q66K14,GGGLTGLLVGTLDVVLDSSAR,1250
0,Q66K14,ILHQTQDSQVYWTVACGSSR,1250
0,Q66K14,HWEWLENNLLQTLSIFDSEEDITTFVK,1250


#### Verification: check if expected number of peptides are present following reformat

In [19]:
fasta_peptides.shape

(556288, 3)

#### Verification: check missed cleavages in code works correctly on FASTA sequence

In [20]:
# KKKK+ should not exist in peptide sequences
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("KKKK", na=False)])

0

In [21]:
# should not allow peptides with "KKK"
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("KKK", na=False)])

0

In [22]:
# should not allow peptides with "KK"
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("KK", na=False)])

0

In [23]:
# RRRR+ should not exist in peptide sequences
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("RRRR", na=False)])

0

In [24]:
# should not allow peptides with "RRR"
len(fasta_peptides[fasta_peptides['Peptide'].str.contains("RRR", na=False)])

0

### 6. Check for total undigested proteins and remove

In [25]:
# export as tsv
fasta_peptides.to_csv("../data/fasta_peptides.tsv", sep='\t', index=False)

In [26]:
peptides = pd.read_table('../data/fasta_peptides.tsv')
peptides.shape

(556288, 3)

In [30]:
# print total number of missing values in fasta peptides dataset
print(peptides.isnull().sum().sum())

# these are proteins that were unable to be digested by trypsin because they lack the target amino acids.

26


In [31]:
# print total number of missing values for each feature
print(peptides.isnull().sum())

Protein     0
Peptide    26
Length      0
dtype: int64


In [32]:
# print all rows with missing values to view in more detail
peptides[peptides.isnull().any(axis=1)]

,Protein,Peptide,Length
19803,Q9BY19,NaN,250
28630,P0DPI4,NaN,4
41993,A0A075B706,NaN,16
87250,Q07326,NaN,219
109962,C9JFL3,NaN,82
126874,P62945,NaN,25
138406,Q6UWW9,NaN,146
151659,Q3MUY2,NaN,71
153292,P60329,NaN,112
175060,O95424,NaN,95


In [33]:
# check if these NaN proteins were correctly undigested

# load dataset
proteins = pd.read_table('../data/fasta_proteins.tsv', header=0)
proteins.head()

,Protein,Sequence
0,Q66K14,MWLSPEEVLVANALWVTERANPFFVLQRRRGHGRGGGLTGLLVGTL...
1,Q9UMR3,MEFTASPKPQLSSRANAFSIAALMSSGGSKEKEATENTIKPLEQFV...
2,Q9P031,MAPVRRSAKWRPGGIEARGEGVSTVGYRNKNVRQKTWRPNHPQAFV...
3,Q6PEY2,MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGD...
4,Q9P016,MSRPRKRLAGTSGSDKGLSGKRTKTENSGEALAKVEDSNPQKTSAT...


In [34]:
# display max column contents
pd.set_option('display.max_colwidth', None)

In [35]:
# check if these NaN proteins were correctly undigested
proteins[proteins['Protein'].str.contains("Q3LI58")]

,Protein,Sequence
18097,Q3LI58,MCCNYYGNSCGYGSGCGCGYGSGSGCGCGYGTGYGCGYGCGFGSHYGCGYGTGYGCGYGSGSGYCGYRPFCFRRCYSSC


In [36]:
proteins.loc[696]

Protein                                                                                                                                                                                                                                                         Q9BY19
Sequence    MNSMTSAVPVANSVLVVAPHNGYPVTPGIMSHVPLYPNSQPQVHLVPGNPPSLVSNVNGQPVQKALKEGKTLGAIQIIIGLAHIGLGSIMATVLVGEYLSISFYGGFPFWGGLWFIISGSLSVAAENQPYSYCLLSGSLGLNIVSAICSAVGVILFITDLSIPHPYAYPDYYPYAWGVNPGMAISGVLLVFCLLEFGIACASSHFGCQLVCCQSSNVSVIYPNIYAANPVITPEPVTSPPSYSSEIQANK
Name: 696, dtype: object

In [39]:
# remove undigested proteins (i.e. 'Peptides' with NaN values)
all_digested_proteins = peptides
all_digested_proteins.dropna(subset = ["Peptide"], inplace=True)
all_digested_proteins.describe(include="all")

,Protein,Peptide,Length
count,556262,556262,556262.000000
unique,20354,526936,NaN
top,Q8WZ42,IHTGEKPYK,NaN
freq,1839,362,NaN
mean,NaN,NaN,1202.077699
std,NaN,NaN,2229.114067
min,NaN,NaN,12.000000
25%,NaN,NaN,451.000000
50%,NaN,NaN,736.000000
75%,NaN,NaN,1278.000000


### 7. Export digested peptide dataset as TSV

In [41]:
# export as tsv
all_digested_proteins.to_csv("../data/fasta_peptides.tsv", sep='\t', index=False)

In [42]:
# final check on dimensions
all_digested_proteins.shape

(556262, 3)